In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

Using TensorFlow backend.


In [3]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [8]:
early_stopping = EarlyStopping(patience=10, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.1, patience=3,verbose=1, monitor="val_acc")

In [10]:
def createModelMobileNetV2():
  base_model = MobileNetV2(weights='imagenet', 
    include_top=False, pooling='avg', 
    input_shape=(224, 224, 3))
    
  x = base_model.output
  x = Dense(512, activation="relu")(x)
  x = Dropout(0.5)(x)
  
  predictions = Dense(120, 
    activation='softmax')(x)

  model = Model(inputs=base_model.input, 
    outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False

  model.compile(optimizer='sgd', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']) 
  
  return model

In [11]:
model = createModelMobileNetV2()

# Using the ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.25],
                                   brightness_range=[0.5, 1.5],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [13]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // BATCH_SIZE,
    epochs = EPOCHS)

Epoch 1/100
481/481 [==============================] - 406s 844ms/step - loss: 4.3959 - acc: 0.0835 - val_loss: 3.2646 - val_acc: 0.3667
Epoch 2/100
481/481 [==============================] - 296s 615ms/step - loss: 3.1826 - acc: 0.2765 - val_loss: 1.9622 - val_acc: 0.5607
Epoch 3/100
481/481 [==============================] - 253s 526ms/step - loss: 2.4085 - acc: 0.4029 - val_loss: 1.4638 - val_acc: 0.6406
Epoch 4/100
481/481 [==============================] - 258s 536ms/step - loss: 2.0228 - acc: 0.4695 - val_loss: 1.2694 - val_acc: 0.6581
Epoch 5/100
481/481 [==============================] - 253s 526ms/step - loss: 1.8242 - acc: 0.5130 - val_loss: 1.1596 - val_acc: 0.6792
Epoch 6/100
481/481 [==============================] - 219s 454ms/step - loss: 1.6986 - acc: 0.5398 - val_loss: 1.1173 - val_acc: 0.6792
Epoch 7/100
481/481 [==============================] - 218s 452ms/step - loss: 1.5933 - acc: 0.5641 - val_loss: 1.0581 - val_acc: 0.6912
Epoch 8/100
481/481 [====================

KeyboardInterrupt: 

## Evaluation

In [30]:
loss, acc = model.evaluate_generator(validation_generator,verbose=0, steps=validation_generator.samples // batch_size)

In [31]:
print(loss,acc)

1.2640717449620014 0.734005905511811


In [32]:
model.save("efficientnetb3-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))